In [8]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct"
quant_name = hub_name + "-AWQ"

# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
dataset_name = "StarkWizard/cairo-instruct"

quant_config = {
    "zero_point":True,
    "q_group_size":128,
    "w_bit":4,
}

In [ ]:
#Loading Model and tokenizer

model = AutoAWQForCausalLM.from_pretrained(hub_name,device_map="auto",)



In [11]:
tokenizer = AutoTokenizer.from_pretrained(hub_name,trust_remote_code=True, use_auth_token=True, add_eos_token=True, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 18610

#quantize
model.quantize(tokenizer=tokenizer,quant_config=quant_config)

#Now we save
model.save_quantized(quant_name, safetensors=True, shard_size="1GB")
tokenizer.save_pretrained(quant_name)

Repo card metadata block was not found. Setting CardData to empty.
AWQ:  59%|█████▉    | 19/32 [1:11:10<48:54, 225.75s/it]

In [ ]:
tokenizer.push_to_hub(quant_name)
model.push_to_hub(quant_name)

In [ ]:
#Pushing Files to HF

from huggingface_hub import HfApi

api = HfApi()
repo = quant_name
local = "./"+repo+"/"
files =[
    local+"config.json",
    local+"generation_config.json",
    local+"quant_config.json",
    local+"tokenizer_config.json",
    local+"special_tokens_map.json",
    local+"tokenizer.json",
] 

for f in files:
    name = f.split("/")[-1]

    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=name,
        repo_id=repo,
        repo_type="model",
    )
    print(f"Uploaded {name}")

In [ ]:
model= AutoAWQForCausalLM.from_quantized(quant_name,device_map="auto",fuse_layers=True,
                trust_remote_code=True,safetensors=True)

tokenizer = AutoTokenizer.from_pretrained(quant_name,trust_remote_code=True, use_auth_token=True, add_eos_token=True, use_fast=False)